In [267]:
using DataFrames, Plots

### Reading in the data
The data comes in comma separated format. I'm using the DataFrames package again, to deal with it.
The method readtable in this package provides the necessary functionality, however, I have to give it a few options.
readtable defaults to the tab separator \t, so I set the separator option to the comma character.
Second, there is no header in each of the files. So I copy the column names from adult.names and store them as an array of symbols in col_names. I can provide column names to the readtable method using the names option.
Additionally, I let readtable now that there is no header and it should not skip the first line.

In the data, missing values are marked with a ?-sign. With the option nastring, I can define which strings readtable should interpret as not available (NA). It defaults to "NA" and the empty string.

Finally, the comments in the files start with the | symbol. I let readtable know to skip these lines using the option commentmark.

In [268]:
col_names = [:age, :wc, :fnlwgt, :edu, :edu_num, :marital, :occ, :rel, :race, :sex, :cap_gain, :cap_loss, :hpw, :nat_ctry, :inc]
adult_train = DataFrames.readtable("adult.data", separator=',', names = col_names, header=false, nastrings=["", "NA", "?"]);
adult_test = DataFrames.readtable("adult.test", separator=',', names = col_names, header=false, allowcomments=true, commentmark='|', nastrings=["", "NA", "?"]);

### Getting rid of lines with missing values
Using the command 'completecases', I am provided with an array of boolean values of the same length as the data set.
If the value at an index in this array is false then the data point with the same index is not complete, i.e. the features of it contain one or more NA values. If the value is true, all features are present.

In [269]:
println("whole training data set: $(size(adult_train, 1))")
println("complete cases: $(size(adult_train[completecases(adult_train),:],1))")

println("whole test data set: $(size(adult_test, 1))")
println("complete cases: $(size(adult_test[completecases(adult_test),:], 1))")

whole training data set: 32561
complete cases: 30162
whole test data set: 16281
complete cases: 15060


With the command 'completecases!', I can remove all incomplete data points from the dataframe.

In [270]:
adult_train = completecases!(adult_train)
adult_test = completecases!(adult_test);


### Utility functions for deriving dummy variables and for normalizing continuous data
Querying a data frame for a certain value in a column can be done via 'df[column_name] .== value'. Similar to the completecases method, this will result in an array of boolean values (being true if the column value is equal to value at a certain index and false otherwise). When querying for multiple values, I simply do the same step over and over again (according to the number of multiple values) and do a logic OR operation on the outcomes. The result will be an array of boolean values that is true where the column value is equal to one of the sought values and zero otherwise.

Normalization is needed, since using gradient methods can result in overshooting, meaning that the gradients that we follow in order to optimize the target function may get very large if the x-values are large already. To overcome this, it is useful to project the feature values to a predefined format. One can e.g. normalize them to lie in the $[0,1]$ interval or as I have done here, do a z-transform. After doing so, the data will have mean zeros and unit variance.

In [271]:
function find_value_idx(df, values, col)
    ret = falses(size(df,1))
    for val in values
        ret |= (df[col] .== val)
    end
    ret
end

function z_normalize(data)
    mu = mean(data)
    (data .- mu) ./ stdm(data, mu)
end

z_normalize (generic function with 1 method)

## Introduce dummy variables
1. workclass
  - wc_1 = 1 if workclass = Private, 0 otherwise
  - wc_2 = 1 if workclass = self-employed(Self-emp-not-inc, Self-emp-inc), 0 otherwise
  - wc_3 = 1 if workclass = government-employed(Federal-gov, Local-gov, State-gov, 0 otherwise
  - if all wc_* are 0, workclass is unemployed(Without-pay, Never-worked)
2. education
  - edu_1 = 1 if education = higher education(Doctorate, Masters, Bachelors), 0 otherwise
  - edu_2 = 1 if education = high school(HS-grad, Some-college, Prof-school, Assoc-adm, Assoc-voc), 0 otherwise
  - if edu_1 = 0 and edu_2 = 0, education is lower education(rest)
3. sex
  - sex = 1 if Female, 0 otherwise
4. response
  - inc = 0 if income <=50K, 1 otherwise

### Prepare the data, including one-padding

In [272]:
X = zeros(size(adult_train,1), 13)
X_test = zeros(size(adult_test,1), 13)
X[:,1] .= 1.0
X_test[:,1] .= 1.0
X[:,2] = z_normalize(Array(adult_train[:age]))
X_test[:,2] = z_normalize(Array(adult_test[:age]))
# working class
wc_1_idx = find_value_idx(adult_train, ["Private"], :wc)
wc_2_idx = find_value_idx(adult_train, ["Self-emp-not-inc","Self-emp-inc"], :wc)
wc_3_idx = find_value_idx(adult_train, ["Federal-gov", "Local-gov", "State-gov"], :wc)
wc_1_idx_test = find_value_idx(adult_test, ["Private"], :wc)
wc_2_idx_test = find_value_idx(adult_test, ["Self-emp-not-inc","Self-emp-inc"], :wc)
wc_3_idx_test = find_value_idx(adult_test, ["Federal-gov", "Local-gov", "State-gov"], :wc)
X[wc_1_idx,3] .= 1.0
X[wc_2_idx,4] .= 1.0
X[wc_3_idx, 5] .= 1.0
X_test[wc_1_idx_test, 3] .= 1.0
X_test[wc_2_idx_test, 4] .= 1.0
X_test[wc_3_idx_test, 5] .= 1.0
X[:,6] = z_normalize(Array(adult_train[:fnlwgt]))
X_test[:, 6] = z_normalize(Array(adult_test[:fnlwgt]))
# education
edu_1_idx = find_value_idx(adult_train, ["Doctorate", "Masters", "Bachelors"], :edu)
edu_2_idx = find_value_idx(adult_train, ["HS-grad", "Some-college", "Prof-school", "Assoc-adm", "Assoc-voc"], :edu)
edu_1_idx_test = find_value_idx(adult_test, ["Doctorate", "Masters", "Bachelors"], :edu)
edu_2_idx_test = find_value_idx(adult_test, ["HS-grad", "Some-college", "Prof-school", "Assoc-adm", "Assoc-voc"], :edu)
X[edu_1_idx, 7] .= 1.0
X[edu_2_idx, 8] .= 1.0
X_test[edu_1_idx_test, 7] .= 1.0
X_test[edu_2_idx_test, 8] .= 1.0
X[:, 9] = z_normalize(Array(adult_train[:edu_num]))
X_test[:, 9] = z_normalize(Array(adult_test[:edu_num]))
X[adult_train[:sex].=="Female", 10] .= 1.0
X_test[adult_test[:sex].=="Female", 10] .= 1.0
X[:, 11] = z_normalize(Array(adult_train[:cap_gain]))
X[:, 12] = z_normalize(Array(adult_train[:cap_loss]))
X[:, 13] = z_normalize(Array(adult_train[:hpw]))
X_test[:, 11] = z_normalize(Array(adult_test[:cap_gain]))
X_test[:, 12] = z_normalize(Array(adult_test[:cap_loss]))
X_test[:, 13] = z_normalize(Array(adult_test[:hpw]));

Y = zeros(size(X,1))
Y_test = zeros(size(X_test,1))
Y[adult_train[:inc] .== ">50K"] = 1.
Y_test[adult_test[:inc] .== ">50K."] = 1.;
sum(Y_test .== 1.)

3700

## KNN clustering
Takes about 1.5min due to the size of training and test set.

In [273]:
n = size(X,1)
n_test = size(X_test,1)
println("Training set size: $n Test set size: $n_test")
println("# of operations needed to compute KNN clustering: $(n*n_test)")
println("time, if one operation takes 1ms: $(n*n_test*2e-7)sec")

Training set size: 30162 Test set size: 15060
# of operations needed to compute KNN clustering: 454239720
time, if one operation takes 1ms: 90.847944sec


## Maximum likelihood computation for logistic regression
First, define functions for computing $h_\theta(x)$ and $\ell(\theta)$.

In [274]:
h(theta, x) = exp(dot(theta,x))/(1+exp(dot(theta,x)))
likelihood(theta, X, Y) = sum([Y[i]*dot(theta,X[i,:]) - log(1 + exp(dot(theta,X[i,:]))) for i in 1:size(X,1)])

likelihood (generic function with 1 method)

### Computing the gradient
The gradient of the log-likelihood has been derived in the lecture and is given by
$$\frac{\partial\ell(\theta)}{\partial\theta} = \sum_{i=1}^n x_i(y_i - h_\theta(x_i))$$

In [275]:
function grad_likelihood(theta, X, Y)
    grad = zeros(length(theta))
    for i in 1:size(X,1)
        grad += X[i,:] .* (Y[i] - h(theta, X[i,:]))
    end
    grad
end

grad_likelihood (generic function with 1 method)

### Utility function
An additional function to be able to compute the accuracy on a given data set, given a parameter estimation of $\theta$.

In [276]:
function compute_accuracy(theta_hat, X_test, Y_test)
    Y_pred = [h(theta_hat, X_test[i,:]) for i in 1:size(X_test,1)]
    sum(round(Y_pred) .== Y_test)/length(Y_test)
end


compute_accuracy (generic function with 1 method)

### Gradient descent
Now implement the gradient descent algorithm. I start with a very low log-likelihood value of $-1*10^100$ and set the step size to a small value of $10^{-5}$ (to prevent overshooting).
While the convergence value (i.e. the relative change in log-likelihood from one iteration to the next) is below a certain threshold ($10^{-3}$ in this example), I keep on computing updates for theta using
$\theta_{i+1} = \theta_i + \gamma\nabla_\theta\ell(\theta_i)$ and recompute the new likelihood value.
Then the convergence value is recomputed using the new value for the log-likelihood.
The rest of the code records and plots training and test set error as a function of algorithm iterations.

In [285]:
function compute_gradient_descent_solution(theta, X, Y)
    old_ll = -1e100
    converge = 1.
    gamma = 1e-5
    train_accs = zeros(0,2)
    while converge > 1e-3
        theta += gamma.*grad_likelihood(theta,X,Y)
        ll = likelihood(theta, X, Y)

        ta = compute_accuracy(theta, X, Y)
        tea = compute_accuracy(theta, X_test, Y_test)
        train_accs = [train_accs; ta tea]
        converge = (old_ll - ll)/old_ll
        old_ll = ll
    end
    display(Plots.plot(collect(1:size(train_accs,1)), train_accs, xlabel="accuracy", ylabel="iteration", labels=["training set", "test set"]))
    theta
end

compute_gradient_descent_solution (generic function with 1 method)

### Outcome
Computing training set and test set accuracies.

In [278]:
train_acc = 0
test_acc = 0
for n in 1:10
    theta_hat = compute_gradient_descent_solution(randn(13), X, Y)
    train_acc += compute_accuracy(theta_hat, X, Y)
    test_acc += compute_accuracy(theta_hat, X_test, Y_test)
end



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 0.65 
 
 
 0.70 
 
 
 0.75 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,362.06 46.2795,335.328 53.189,313.593 60.0984,294.812 67.0079,276.713 73.9173,261.416 80.8268,246.421 87.7362,231.048 94.6457,217.644 101.555,205.603 
 108.465,194.244 115.374,182.884 122.283,172.812 129.193,162.892 136.102,155.092 143.012,144.263 149.921,138.583 156.831,132.222 163.74,126.618 170.65,122.074 
 177.559,118.893 184.469,113.744 191.378,108.367 198.287,103.596 205.197,98.0676 212.106,92.6909 219.016,89.2073 225.925,85.7238 232.835,80.9528 239.744,76.0304 
 246.654,71.108 253.563,66.9429 260.472,63.3836 267.382,60.7331 274.291,56.0379 281.201,54.2961 288.11,51.3427 295.02,48.9193 301.929,45.8144 308.839,43.3911 
 315.748,41.725 322.657,40.2862 329.567,38.6201 336.476,37.4085 343.386,35.8939 350.295,34.6065 357.205,32.789 364.114,32.0317 371.024,30.6685 377.933,29.987 
 384.843,28.6239 391.752,27.3365 398.661,27.185 405.571,25.5189 412.48,24.9131 419.39,23.7014 426.299,22.7927 433.209,22.0354 440.118,21.3538 447.028,20.2179 
 453.937,19.3091 460.846,18.4761 467.756,17.7188 474.665,16.3557 481.575,16.28 488.484,16.1285 495.394,14.7654 502.303,14.3867 509.213,13.4023 516.122,12.7964 
 523.031,12.5692 529.941,11.8119 536.85,11.2818 543.76,10.8275 550.669,10.1459 557.579,9.54006 564.488,9.31287 571.398,9.08569 578.307,9.00996 585.217,8.17693 
 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 46.2795,346.057 53.189,321.031 60.0984,297.826 67.0079,280.384 73.9173,266.278 80.8268,248.381 87.7362,236.551 94.6457,223.811 101.555,210.767 
 108.465,197.572 115.374,185.438 122.283,174.67 129.193,167.39 136.102,159.958 143.012,152.526 149.921,144.336 156.831,137.511 163.74,132.202 170.65,127.046 
 177.559,118.4 184.469,114.305 191.378,110.514 198.287,107.48 205.197,103.84 212.106,98.6833 219.016,95.9532 225.925,90.7964 232.835,86.2463 239.744,82.7579 
 246.654,77.7528 253.563,73.6577 260.472,68.8043 267.382,62.8892 274.291,59.8558 281.201,57.8841 288.11,55.0023 295.02,52.7273 301.929,50.7556 308.839,48.1772 
 315.748,46.5088 322.657,44.6888 329.567,44.5371 336.476,43.1721 343.386,41.2004 350.295,40.1387 357.205,38.0153 364.114,36.9536 371.024,35.2853 377.933,34.2236 
 384.843,33.7686 391.752,32.8585 398.661,31.9485 405.571,30.5835 412.48,29.2185 419.39,27.7018 426.299,26.3367 433.209,25.8817 440.118,25.275 447.028,25.4267 
 453.937,24.365 460.846,23 467.756,23 474.665,22.6967 481.575,22.6967 488.484,21.9383 495.394,21.7866 502.303,21.0283 509.213,21.0283 516.122,20.2699 
 523.031,19.5116 529.941,19.8149 536.85,19.2083 543.76,18.6016 550.669,19.2083 557.579,18.4499 564.488,17.6916 571.398,17.0849 578.307,16.0232 585.217,15.5682 
 592.126,15.4165 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 0.70 
 
 
 0.75 
 
 
 0.80 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 45.5808,348.407 51.7916,335.352 58.0023,322.641 64.213,310.102 70.4238,296.704 76.6345,287.601 82.8453,276.951 89.056,269.135 95.2667,260.547 
 101.477,251.014 107.688,242.769 113.899,233.837 120.11,226.537 126.32,219.065 132.531,214.169 138.742,208.587 144.953,204.035 151.163,199.053 157.374,193.471 
 163.585,188.06 169.796,182.821 176.006,176.637 182.217,171.399 188.428,168.221 194.639,164.957 200.849,158.773 207.06,153.878 213.271,150.271 219.482,144.087 
 225.692,139.621 231.903,133.695 238.114,129.229 244.325,123.99 250.535,119.009 256.746,114.199 262.957,107.844 269.167,102.605 275.378,97.6235 281.589,93.1575 
 287.8,88.1762 294.01,84.3114 300.221,79.9313 306.432,77.4406 312.643,74.2629 318.853,71.4287 325.064,70.0545 331.275,68.8522 337.486,66.018 343.696,63.6132 
 349.907,63.6132 356.118,62.0673 362.329,59.9202 368.539,57.9448 374.75,56.1412 380.961,53.5647 387.172,52.1905 393.382,49.6999 399.593,47.5528 405.804,46.1786 
 412.015,44.8903 418.225,42.3138 424.436,41.3691 430.647,40.9396 436.857,39.3937 443.068,36.6454 449.279,34.7559 455.49,33.5535 461.7,32.6088 467.911,30.6335 
 474.122,29.8605 480.333,27.5416 486.543,25.9098 492.754,24.3639 498.965,23.9345 505.176,21.7873 511.386,21.4438 517.597,19.8979 523.808,18.0084 530.019,17.4931 
 536.229,16.8919 542.44,15.5178 548.651,13.2848 554.862,12.34 561.072,11.4812 567.283,10.7941 573.494,10.7082 579.705,9.93525 585.915,9.16229 592.126,7.87402 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,363.965 45.5808,346.076 51.7916,330.595 58.0023,313.394 64.213,302.558 70.4238,292.237 76.6345,284.497 82.8453,275.208 89.056,265.92 95.2667,257.663 
 101.477,249.923 107.688,239.602 113.899,229.798 120.11,226.53 126.32,221.025 132.531,216.381 138.742,212.081 144.953,207.265 151.163,204.34 157.374,197.116 
 163.585,190.752 169.796,184.387 176.006,178.367 182.217,174.239 188.428,168.907 194.639,165.982 200.849,161.166 207.06,156.35 213.271,151.19 219.482,143.965 
 225.692,135.881 231.903,132.957 238.114,128.14 244.325,122.292 250.535,116.788 256.746,113.348 262.957,106.123 269.167,101.823 275.378,97.8669 281.589,93.2227 
 287.8,91.8466 294.01,87.5464 300.221,84.4502 306.432,80.666 312.643,77.5698 318.853,76.3658 325.064,73.2696 331.275,70.6895 337.486,69.8294 343.696,66.9053 
 349.907,64.4972 356.118,62.6051 362.329,59.1649 368.539,58.4768 374.75,57.1008 380.961,53.8326 387.172,51.4245 393.382,50.5644 399.593,47.9843 405.804,45.5762 
 412.015,43.168 418.225,40.5879 424.436,39.7279 430.647,39.8999 436.857,39.2118 443.068,38.3518 449.279,36.8037 455.49,36.2877 461.7,34.5676 467.911,32.8475 
 474.122,30.9554 480.333,29.5793 486.543,28.3753 492.754,25.1071 498.965,23.387 505.176,22.699 511.386,22.355 517.597,19.2588 523.808,18.5708 530.019,17.1947 
 536.229,15.4746 542.44,15.3026 548.651,15.3026 554.862,14.4426 561.072,13.4105 567.283,13.4105 573.494,13.0665 579.705,13.2385 585.915,11.0024 592.126,9.28229 
 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0.76 
 
 
 0.78 
 
 
 0.80 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,349.679 47.4988,338.959 55.6276,328.868 63.7564,320.67 71.8851,309.949 80.0139,296.496 88.1427,285.775 96.2714,277.366 104.4,268.537 112.529,256.765 
 120.658,248.988 128.786,242.261 136.915,235.744 145.044,227.126 153.173,218.717 161.302,214.513 169.43,208.417 177.559,201.9 185.688,196.014 193.817,189.918 
 201.945,185.924 210.074,179.618 218.203,173.732 226.332,170.158 234.46,164.903 242.589,159.647 250.718,153.972 258.847,148.296 266.975,145.353 275.104,142.62 
 283.233,138.626 291.362,133.371 299.491,129.587 307.619,124.962 315.748,117.184 323.877,111.509 332.006,103.521 340.134,97.4245 348.263,91.7488 356.392,86.4935 
 364.521,81.4484 372.649,76.193 380.778,72.8296 388.907,67.9948 397.036,63.7905 405.164,57.4841 413.293,53.9105 421.422,51.1778 429.551,49.9165 437.679,45.502 
 445.808,41.2978 453.937,39.4059 462.066,37.3038 470.195,36.4629 478.323,33.9404 486.452,32.4689 494.581,29.7361 502.71,27.2136 510.838,25.5319 518.967,24.9012 
 527.096,21.9582 535.225,19.6459 543.353,17.3336 551.482,13.9702 559.611,12.9191 567.74,11.2374 575.868,10.817 583.997,8.08423 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 47.4988,357.137 55.6276,343.243 63.7564,330.192 71.8851,316.299 80.0139,301.142 88.1427,292.722 96.2714,285.144 104.4,276.303 112.529,272.513 
 120.658,268.303 128.786,259.041 136.915,252.726 145.044,246.411 153.173,238.833 161.302,230.412 169.43,224.518 177.559,212.73 185.688,211.046 193.817,208.099 
 201.945,203.468 210.074,197.994 218.203,196.31 226.332,193.363 234.46,188.311 242.589,181.154 250.718,176.944 258.847,171.892 266.975,168.524 275.104,164.314 
 283.233,160.945 291.362,158.84 299.491,155.893 307.619,146.631 315.748,139.474 323.877,131.896 332.006,123.897 340.134,118.423 348.263,110.845 356.392,107.056 
 364.521,102.846 372.649,96.1098 380.778,91.4787 388.907,86.0055 397.036,81.3744 405.164,79.6904 413.293,77.1643 421.422,74.2172 429.551,69.1651 437.679,64.113 
 445.808,59.9028 453.937,55.6927 462.066,55.6927 470.195,51.9036 478.323,49.3776 486.452,43.0624 494.581,41.7994 502.71,41.3784 510.838,41.7994 518.967,38.4313 
 527.096,37.5893 535.225,35.9052 543.353,36.3262 551.482,34.6422 559.611,33.8002 567.74,34.6422 575.868,35.0632 583.997,32.5371 592.126,32.1161 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0.725 
 
 
 0.750 
 
 
 0.775 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,360.962 46.5487,346.866 53.7274,333.26 60.906,318.918 68.0847,307.274 75.2633,294.403 82.442,281.042 89.6206,269.643 96.7993,261.063 103.978,252.482 
 111.157,246.721 118.335,241.45 125.514,232.87 132.693,225.516 139.871,219.877 147.05,213.993 154.228,208.11 161.407,202.349 168.586,192.788 175.764,184.453 
 182.943,178.569 190.122,172.318 197.3,163.737 204.479,157.118 211.658,147.802 218.836,141.061 226.015,133.584 233.194,126.965 240.372,120.836 247.551,116.3 
 254.73,112.991 261.908,110.049 269.087,105.514 276.265,99.8753 283.444,97.0561 290.623,93.3788 297.801,88.5983 304.98,83.4502 312.159,80.1406 319.337,77.0762 
 326.516,74.6247 333.695,70.5797 340.873,68.8636 348.052,64.9412 355.231,62.98 362.409,59.5479 369.588,56.2383 376.767,55.1351 383.945,53.0514 391.124,51.2127 
 398.302,49.987 405.481,47.658 412.66,46.9226 419.838,45.2065 427.017,43.1227 434.196,41.6518 441.374,40.1809 448.553,39.2003 455.732,37.2391 462.91,35.2779 
 470.089,34.0521 477.268,33.4392 484.446,32.0909 491.625,30.1297 498.804,28.291 505.982,27.5556 513.161,25.717 520.34,25.1041 527.518,24.0009 534.697,23.388 
 541.875,22.2848 549.054,21.9171 556.233,21.7945 563.411,20.3236 570.59,19.343 577.769,17.3818 584.947,16.0335 592.126,15.298 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 46.5487,352.792 53.7274,337.326 60.906,324.561 68.0847,310.322 75.2633,300.502 82.442,289.21 89.6206,279.144 96.7993,270.307 103.978,261.96 
 111.157,253.122 118.335,245.512 125.514,237.901 132.693,228.327 139.871,222.19 147.05,215.807 154.228,209.915 161.407,203.041 168.586,195.186 175.764,188.066 
 182.943,180.947 190.122,174.319 197.3,164.99 204.479,154.679 211.658,147.56 218.836,140.686 226.015,133.321 233.194,125.711 240.372,119.819 247.551,116.137 
 254.73,109.508 261.908,104.844 269.087,99.9341 276.265,95.2697 283.444,90.3598 290.623,86.6774 297.801,83.7315 304.98,78.3306 312.159,75.6302 319.337,71.4568 
 326.516,67.5289 333.695,67.7744 340.873,64.3375 348.052,60.6551 355.231,57.4637 362.409,53.5358 369.588,50.5899 376.767,50.5899 383.945,48.3804 391.124,45.9255 
 398.302,45.4345 405.481,42.7341 412.66,41.5066 419.838,39.7882 427.017,37.3332 434.196,35.6148 441.374,32.9144 448.553,32.1779 455.732,30.2139 462.91,29.9684 
 470.089,29.2319 477.268,27.759 484.446,26.5315 491.625,26.286 498.804,23.3401 505.982,22.8491 513.161,21.1307 520.34,20.1487 527.518,18.9212 534.697,17.4483 
 541.875,15.2388 549.054,14.0114 556.233,13.2749 563.411,11.3109 570.59,11.0654 577.769,10.329 584.947,8.85599 592.126,7.87402 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 49.0676,345.291 58.765,314.462 68.4625,277.404 78.16,240.216 87.8574,206.404 97.5549,174.109 107.252,152.283 116.95,135.115 126.647,120.983 
 136.345,108.892 146.042,99.4185 155.74,90.9655 165.437,82.1198 175.135,74.0331 184.832,67.0194 194.53,60.6338 204.227,55.295 213.925,49.9562 223.622,45.8475 
 233.32,42.3929 243.017,39.6974 252.714,37.1588 262.412,35.4839 272.109,33.1024 281.807,30.3807 291.504,28.5226 301.202,26.9785 310.899,25.2774 320.597,24.1783 
 330.294,22.6865 339.992,21.535 349.689,19.9386 359.387,18.7086 369.084,17.4786 378.782,17.1384 388.479,16.5103 398.177,15.8822 407.874,15.2018 417.571,14.5213 
 427.269,13.6577 436.966,12.9773 446.664,12.4015 456.361,11.9304 466.059,11.8258 475.756,11.3547 485.454,10.8313 495.151,10.334 504.849,10.1247 514.546,9.81064 
 524.244,9.49659 533.941,9.33957 543.639,9.05169 553.336,8.94701 563.034,8.65913 572.731,8.34509 582.429,7.9787 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,366.075 49.0676,345.319 58.765,314.971 68.4625,280.85 78.16,242.797 87.8574,210.667 97.5549,178.485 107.252,157.257 116.95,137.917 126.647,123.398 
 136.345,112.024 146.042,101.699 155.74,91.7923 165.437,83.406 175.135,75.6488 184.832,68.9398 194.53,63.017 204.227,56.2556 213.925,50.49 223.622,45.5631 
 233.32,41.8417 243.017,39.2734 252.714,36.3382 262.412,33.8224 272.109,32.1451 281.807,30.3106 291.504,29.4196 301.202,28.2141 310.899,26.8513 320.597,25.5934 
 330.294,24.5451 339.992,23.2347 349.689,21.5575 359.387,21.243 369.084,20.7188 378.782,19.723 388.479,18.465 398.177,17.6264 407.874,17.4168 417.571,17.0499 
 427.269,16.6306 436.966,16.1064 446.664,16.054 456.361,16.2112 466.059,16.0016 475.756,15.3202 485.454,15.5299 495.151,15.2154 504.849,14.6388 514.546,14.3243 
 524.244,14.2719 533.941,14.0098 543.639,13.7478 553.336,13.2761 563.034,13.014 572.731,12.4374 582.429,12.1754 592.126,11.8085 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0.70 
 
 
 0.75 
 
 
 0.80 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 48.2855,333.551 57.2009,303.532 66.1163,275.981 75.0318,253.775 83.9472,231.405 92.8626,214.545 101.778,200.482 110.693,189.05 119.609,178.358 
 128.524,170.052 137.44,163.884 146.355,157.222 155.271,152.616 164.186,147.024 173.101,142.5 182.017,137.401 190.932,132.385 199.848,126.957 208.763,120.871 
 217.678,115.772 226.594,109.603 235.509,104.258 244.425,99.8164 253.34,96.1155 262.256,91.0987 271.171,88.4669 280.086,82.3809 289.002,77.3641 297.917,73.0052 
 306.833,69.9623 315.748,66.2613 324.663,61.6557 333.579,58.4482 342.494,55.1585 351.41,54.0071 360.325,51.3754 369.241,49.3193 378.156,46.852 387.071,44.5492 
 395.987,41.9997 404.902,39.3679 413.818,35.667 422.733,33.0352 431.648,31.1436 440.564,30.8969 449.479,30.6502 458.395,29.581 467.31,27.196 476.226,26.0445 
 485.141,25.4688 494.056,24.5642 502.972,22.5903 511.887,21.2745 520.803,19.5474 529.718,19.3829 538.633,19.4651 547.549,18.396 556.464,17.8203 565.38,17.1623 
 574.295,16.0109 583.211,15.4352 592.126,15.1062 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,358.882 48.2855,322.151 57.2009,288.219 66.1163,258.57 75.0318,235.016 83.9472,216.239 92.8626,196.308 101.778,183.46 110.693,174.236 119.609,162.871 
 128.524,157.106 137.44,147.388 146.355,143.764 155.271,139.646 164.186,135.199 173.101,129.599 182.017,125.975 190.932,122.68 199.848,117.904 208.763,112.798 
 217.678,110.492 226.594,107.033 235.509,101.267 244.425,95.6672 253.34,90.2316 262.256,87.4314 271.171,82.6547 280.086,77.2191 289.002,73.4306 297.917,70.7952 
 306.833,67.6656 315.748,62.5594 324.663,59.1004 333.579,55.1473 342.494,53.6648 351.41,51.0294 360.325,49.3822 369.241,46.4174 378.156,43.7819 387.071,39.9935 
 395.987,39.664 404.902,39.0052 413.818,35.052 422.733,33.5696 431.648,31.593 440.564,28.7928 449.479,26.8163 458.395,25.6633 467.31,25.0044 476.226,24.675 
 485.141,22.3689 494.056,21.5454 502.972,20.5571 511.887,17.9216 520.803,16.9333 529.718,16.2745 538.633,15.6156 547.549,14.2979 556.464,12.6508 565.38,11.1683 
 574.295,9.35645 583.211,9.35645 592.126,7.87402 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,365.901 51.6535,305.084 63.937,251.622 76.2205,209.301 88.5039,172.478 100.787,143.396 113.071,118.051 125.354,97.8431 137.638,82.241 149.921,69.7496 
 162.205,60.996 174.488,53.6652 186.772,46.1655 199.055,40.3057 211.339,36.2785 223.622,32.0343 235.906,28.8753 248.189,25.3546 260.472,23.2566 272.756,20.8693 
 285.039,19.0848 297.323,17.5897 309.606,16.818 321.89,15.7811 334.173,15.1541 346.457,14.3583 358.74,13.7314 371.024,13.2491 383.307,12.6462 395.591,12.1639 
 407.874,11.8745 420.157,11.6575 432.441,11.2234 444.724,10.9099 457.008,10.6206 469.291,10.5 481.575,10.1383 493.858,9.87301 506.142,9.6801 518.425,9.43895 
 530.709,9.24603 542.992,8.93254 555.276,8.6914 567.559,8.37791 579.843,8.08853 592.126,7.94384 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 51.6535,306.974 63.937,255.007 76.2205,211.782 88.5039,174.159 100.787,144.843 113.071,123.496 125.354,106.447 137.638,90.7025 149.921,77.9522 
 162.205,68.1963 174.488,58.8268 186.772,52.2102 199.055,46.1248 211.339,40.0395 223.622,35.2098 235.906,31.4427 248.189,27.3858 260.472,25.1159 272.756,22.6527 
 285.039,20.2862 297.323,17.968 309.606,16.664 321.89,15.36 334.173,14.4423 346.457,13.573 358.74,13.09 371.024,12.9451 383.307,12.3656 395.591,12.0275 
 407.874,12.0758 420.157,11.5445 432.441,11.2065 444.724,10.6269 457.008,10.2405 469.291,9.90247 481.575,9.75758 493.858,9.32291 506.142,9.12972 518.425,9.03313 
 530.709,8.79165 542.992,8.55017 555.276,8.50187 567.559,8.35698 579.843,8.01891 592.126,7.87402 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 48.0069,288.122 56.6437,189.451 65.2805,131.438 73.9173,111.873 82.5541,98.8805 91.1909,92.9297 99.8278,88.3809 108.465,83.1779 117.101,79.6573 
 125.738,76.6352 134.375,74.2362 143.012,71.6191 151.649,68.6904 160.285,66.1668 168.922,64.2039 177.559,61.8049 186.196,59.5929 194.833,57.4743 203.469,55.5114 
 212.106,53.7667 220.743,51.3989 229.38,49.4983 238.017,47.7848 246.654,45.7285 255.29,43.8591 263.927,41.8651 272.564,39.3415 281.201,37.0983 289.838,34.9485 
 298.474,33.0169 307.111,30.649 315.748,29.1535 324.385,27.8761 333.022,26.6922 341.658,25.7575 350.295,24.9163 358.932,23.2962 367.569,22.0188 376.206,20.7414 
 384.843,19.6198 393.479,18.9344 402.116,18.3424 410.753,17.657 419.39,16.7846 428.027,16.255 436.663,15.8188 445.3,15.2268 453.937,14.7283 462.574,14.261 
 471.211,13.8248 479.847,13.2017 488.484,12.7343 497.121,12.1424 505.758,11.4881 514.395,11.2077 523.031,10.7092 531.668,10.3665 540.305,9.868 548.942,9.52528 
 557.579,9.40066 566.216,9.15141 574.852,8.93332 583.489,8.46598 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,366.778 48.0069,287.282 56.6437,189.566 65.2805,131.41 73.9173,112.067 82.5541,101.272 91.1909,92.2239 99.8278,87.7312 108.465,82.6769 117.101,78.933 
 125.738,76.0626 134.375,73.1299 143.012,69.5108 151.649,67.202 160.285,64.0197 168.922,61.1493 177.559,60.2757 186.196,57.1558 194.833,54.0983 203.469,51.3527 
 212.106,48.9816 220.743,47.2968 229.38,44.9881 238.017,43.5529 246.654,41.3065 255.29,40.0586 263.927,38.4986 272.564,36.3146 281.201,34.7547 289.838,32.8203 
 298.474,31.5099 307.111,30.1372 315.748,29.2636 324.385,27.6412 333.022,25.582 341.658,24.8957 350.295,23.2109 358.932,22.5869 367.569,21.3389 376.206,20.7149 
 384.843,20.0286 393.479,19.5294 402.116,18.3438 410.753,17.7822 419.39,17.1582 428.027,16.5966 436.663,15.6606 445.3,15.411 453.937,14.7871 462.574,14.2879 
 471.211,13.9759 479.847,13.4143 488.484,12.9151 497.121,12.4783 505.758,11.9167 514.395,11.9167 523.031,11.4175 531.668,11.4175 540.305,11.1679 548.942,11.0431 
 557.579,10.4815 566.216,9.73276 574.852,9.60796 583.489,8.92158 592.126,8.79678 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 47.0472,342.914 54.7244,316.583 62.4016,291.556 70.0787,269.55 77.7559,249.824 85.4331,232.616 93.1102,218.192 100.787,204.39 108.465,190.914 
 116.142,180.784 123.819,169.648 131.496,158.837 139.173,150.87 146.85,144.265 154.528,136.002 162.205,129.871 169.882,123.266 177.559,116.335 185.236,109.583 
 192.913,103.481 200.591,98.8312 208.268,92.8483 215.945,87.5467 223.622,82.8374 231.299,78.3651 238.976,73.8039 246.654,70.1905 254.331,65.9847 262.008,62.3417 
 269.685,58.284 277.362,55.1741 285.039,51.5311 292.717,48.8951 300.394,46.2294 308.071,44.245 315.748,42.1125 323.425,39.9504 331.102,37.9364 338.78,36.5147 
 346.457,34.7969 354.134,32.7532 361.811,30.3245 369.488,28.3105 377.165,26.7407 384.843,24.7267 392.52,22.7719 400.197,21.4687 407.874,20.4913 415.551,19.0104 
 423.228,18.0626 430.906,17.263 438.583,16.3744 446.26,15.3674 453.937,15.0416 461.614,14.6566 469.291,13.9457 476.969,13.5607 484.646,13.2941 492.323,12.5833 
 500,11.9909 507.677,11.517 515.354,11.1616 523.031,10.8654 530.709,10.5396 538.386,10.2138 546.063,10.1546 553.74,10.0361 561.417,9.6511 569.094,9.3253 
 576.772,8.88103 584.449,8.46638 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,367.954 47.0472,339.599 54.7244,315.101 62.4016,289.238 70.0787,268.595 77.7559,251.274 85.4331,235.554 93.1102,220.072 100.787,205.598 108.465,193.438 
 116.142,182.226 123.819,170.956 131.496,160.99 139.173,153.635 146.85,146.279 154.528,138.924 162.205,132.992 169.882,125.162 177.559,119.052 185.236,111.993 
 192.913,106.061 200.591,101.138 208.268,95.8583 215.945,90.2823 223.622,85.7148 231.299,80.7913 238.976,76.1644 246.654,72.6646 254.331,69.4614 262.008,66.1396 
 269.685,62.0466 277.362,58.9027 285.039,56.5892 292.717,53.5047 300.394,48.9964 308.071,46.8016 315.748,44.5475 323.425,42.2934 331.102,40.0393 338.78,37.1327 
 346.457,35.1751 354.134,33.099 361.811,31.0228 369.488,28.9467 377.165,26.8705 384.843,24.9723 392.52,23.43 400.197,22.1843 407.874,21.1166 415.551,20.1082 
 423.228,19.0998 430.906,18.6252 438.583,17.3202 446.26,16.2525 453.937,15.7779 461.614,15.5406 469.291,15.1254 476.969,14.2356 484.646,13.7611 492.323,13.3458 
 500,12.8713 507.677,12.1001 515.354,11.6849 523.031,11.507 530.709,10.9731 538.386,10.8544 546.063,10.4392 553.74,10.024 561.417,9.84603 569.094,9.31216 
 576.772,8.83761 584.449,8.54101 592.126,8.48169 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0.76 
 
 
 0.78 
 
 
 0.80 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 47.874,364.997 56.378,359.426 64.8819,354.269 73.3858,347.46 81.8898,339.414 90.3937,332.4 98.8976,327.655 107.402,323.322 115.906,321.259 
 124.409,317.133 132.913,313.213 141.417,310.118 149.921,302.072 158.425,296.502 166.929,292.169 175.433,289.487 183.937,284.123 192.441,278.553 200.945,265.143 
 209.449,254.827 217.953,235.434 226.457,223.055 234.961,202.424 243.465,188.395 251.969,174.16 260.472,161.575 268.976,148.784 277.48,136.405 285.984,128.978 
 294.488,117.012 302.992,108.553 311.496,94.9368 320,91.8422 328.504,86.8908 337.008,80.4951 345.512,75.75 354.016,69.9733 362.52,66.0534 371.024,62.9588 
 379.528,58.8326 388.031,54.5001 396.535,52.6433 405.039,48.9297 413.543,46.6603 422.047,41.9152 430.551,38.2016 439.055,35.3132 447.559,33.4565 456.063,30.7744 
 464.567,30.3618 473.071,29.5366 481.575,28.7113 490.079,25.2041 498.583,21.9031 507.087,20.2526 515.591,18.8084 524.094,18.6021 532.598,16.539 541.102,15.9201 
 549.606,16.3327 558.11,14.0633 566.614,11.3813 575.118,8.69926 583.622,7.87402 592.126,8.28664 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,353.13 47.874,347.346 56.378,342.387 64.8819,338.255 73.3858,329.578 81.8898,322.141 90.3937,313.05 98.8976,313.464 107.402,310.158 115.906,308.092 
 124.409,306.026 132.913,305.2 141.417,301.068 149.921,299.415 158.425,288.672 166.929,286.606 175.433,280.408 183.937,284.127 192.441,278.342 200.945,268.839 
 209.449,252.311 217.953,237.849 226.457,222.561 234.961,199.422 243.465,194.877 251.969,182.481 260.472,174.63 268.976,165.54 277.48,152.317 285.984,144.467 
 294.488,133.724 302.992,123.394 311.496,109.758 320,97.3624 328.504,90.7513 337.008,82.9006 345.512,82.0742 354.016,74.6367 362.52,71.7443 371.024,68.4387 
 379.528,66.3728 388.031,56.8693 396.535,56.8693 405.039,56.0429 413.543,53.5637 422.047,53.1505 430.551,49.845 439.055,47.3658 447.559,44.8866 456.063,45.2998 
 464.567,45.2998 473.071,43.647 481.575,41.9942 490.079,39.9283 498.583,36.6227 507.087,37.4491 515.591,35.7963 524.094,34.1435 532.598,33.7303 541.102,34.5567 
 549.606,33.7303 558.11,30.838 566.614,29.1852 575.118,27.9456 583.622,27.5324 592.126,25.4664 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

In [283]:
println("training set accuracay: $(train_acc/10), test set accuracy: $(test_acc/10)")


training set accuracay: 0.8020091505868312, test set accuracy: 0.8008034528552456


### Optional: Newton-Raphson steps
The Newton-Raphson step is given by $$\theta_{\text{new}} = \theta_{\text{old}} - \gamma H^{-1}\nabla_\theta \ell(\theta)$$ with $H$ the Hessian matrix of second derivatives with respect to $\theta$ which can also be written as $$H = \nabla_\theta^2 \ell(\theta) = \frac{\partial \ell(\theta)}{\partial\theta\partial\theta^\top}$$.

We have derived $$\frac{\partial\ell(\theta)}{\partial\theta} = \sum_{i=1}^n x_i(y_i - h_\theta(x_i))\text{ with } h_\theta(x_i) = \frac{\exp{\{\theta^\top x_i\}}}{1+\exp{\{\theta^\top x_i\}}}$$ and used it in the gradient descent algorithm above.

The matrix of second derivatives is given by $$\frac{\partial \ell(\theta)}{\partial\theta\partial\theta^\top} = -\sum_{i=1}^n x_i x_i^\top h_\theta(x_i)(1-h_\theta(x_i))$$

Implementation follows along the lines if gradient descent above. Additionally, we compute the Hessian in each step and the update step for $\theta$ is different.
Notice that computing the Hessian involves a quadratic form in $x_i$ (which is positive) and additional factors that lie between $0$ and $1$ (again positive). The minus sign in front makes the Hessian negative everywhere, so we are guarenteed to run into the direction of a maximum this way.

Also, these updates are more stable (because we take the curvature of the gradient into account) and so we can set the prefactor $\gamma$ to a higher value.

In [284]:
function compute_newton_raphson_solution(theta, X, Y)
    old_ll = -1e100
    converge = 1.
    gamma = .1
    train_accs = zeros(0,2)
    while converge > 1e-3
        ll = likelihood(theta, X, Y)
        H = zeros(size(X,2), size(X,2))
        for i in 1:size(X,1)
            h_theta = h(theta, X[i,:])
            H -= X[i,:]*X[i,:]' * h_theta *(1-h_theta)
        end
        theta -= gamma.*inv(H)*grad_likelihood(theta,X,Y)
        ta = compute_accuracy(theta, X, Y)
        tea = compute_accuracy(theta, X_test, Y_test)
        train_accs = [train_accs; ta tea]
        converge = (old_ll - ll)/old_ll
        old_ll = ll
    end
    display(Plots.plot(collect(1:size(train_accs,1)), train_accs, xlabel="accuracy", ylabel="iteration", labels=["training set", "test set"]))
    println("final accuracies: train: $(train_accs[end,1]), test: $(train_accs[end,2])")
    theta
end

compute_newton_raphson_solution (generic function with 1 method)

We can now go on and compare to the gradient descent solution. The macro @timed measures the time of execution of a command and returns _outcome, time, allocated bytes, garbage collection time and number of memory allocations_ and is a utility that comes with the julia core.

In [286]:
accuracies = zeros(0,4)
exec_times = zeros(0,2)
for n in 1:10
    theta_guess = randn(13)
    theta_hat_gs, gs_time, _, _, _ = @timed compute_gradient_descent_solution(theta_guess, X, Y)
    theta_hat_nr, nr_time, _, _, _ = @timed compute_newton_raphson_solution(theta_guess, X, Y)
    exec_times = [exec_times; gs_time nr_time]
    
    train_acc_gs = compute_accuracy(theta_hat_gs, X, Y)
    train_acc_nr = compute_accuracy(theta_hat_nr, X, Y)
    test_acc_gs = compute_accuracy(theta_hat_gs, X_test, Y_test)
    test_acc_nr = compute_accuracy(theta_hat_nr, X_test, Y_test)
    accuracies = [accuracies; train_acc_gs train_acc_nr test_acc_gs test_acc_nr]
end



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.65 
 
 
 0.70 
 
 
 0.75 
 
 
 accuracy 
 
 
 iteration 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,358.822 44.6344,323.21 49.8988,304.167 55.1631,291.572 60.4274,280.026 65.6918,269.155 70.9561,260.008 76.2205,253.335 81.4848,248.462 86.7492,243.664 
 92.0135,239.915 97.2778,234.742 102.542,229.944 107.807,225.97 113.071,223.646 118.335,219.748 123.6,215.849 128.864,211.65 134.128,208.427 139.393,205.203 
 144.657,201.604 149.921,198.455 155.186,193.807 160.45,189.833 165.714,186.085 170.979,183.761 176.243,179.562 181.507,175.888 186.772,172.215 192.036,169.216 
 197.3,164.792 202.565,160.219 207.829,153.996 213.093,148.673 218.358,144.175 223.622,140.126 228.886,135.478 234.151,130.53 239.415,125.656 244.679,122.658 
 249.944,118.459 255.208,115.085 260.472,113.136 265.737,110.587 271.001,106.613 276.265,103.465 281.53,100.541 286.794,97.9915 292.058,96.3421 297.323,92.5934 
 302.587,89.5945 307.852,85.9208 313.116,82.9219 318.38,79.848 323.645,78.2736 328.909,76.0244 334.173,73.9252 339.438,71.526 344.702,68.5271 349.966,66.053 
 355.231,63.7288 360.495,62.7542 365.759,60.2051 371.024,57.1312 376.288,55.8567 381.552,53.0827 386.817,51.2084 392.081,49.484 397.345,46.785 402.61,45.2855 
 407.874,43.786 413.138,41.9117 418.403,40.1124 423.667,39.2877 428.931,37.7882 434.196,35.9889 439.46,33.9646 444.724,32.84 449.989,30.8907 455.253,29.3912 
 460.517,28.7915 465.782,27.367 471.046,26.5423 476.31,25.9425 481.575,25.4927 486.839,23.9932 492.103,22.8686 497.368,21.4441 502.632,20.2445 507.897,19.7947 
 513.161,18.5202 518.425,18.4452 523.69,17.9954 528.954,17.1707 534.218,16.196 539.483,14.8465 544.747,14.0218 550.011,13.347 555.276,13.1971 560.54,12.5973 
 565.804,12.1475 571.069,11.0229 576.333,10.2731 581.597,9.74834 586.862,8.92364 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 44.6344,333.368 49.8988,309.343 55.1631,291.625 60.4274,279.162 65.6918,268.351 70.9561,259.191 76.2205,253.335 81.4848,244.176 86.7492,237.269 
 92.0135,231.412 97.2778,228.409 102.542,224.806 107.807,220.601 113.071,216.547 118.335,212.793 123.6,208.889 128.864,205.586 134.128,203.484 139.393,199.279 
 144.657,197.027 149.921,194.775 155.186,190.72 160.45,186.366 165.714,183.363 170.979,178.708 176.243,176.306 181.507,172.852 186.772,168.498 192.036,166.245 
 197.3,161.29 202.565,157.686 207.829,150.629 213.093,145.224 218.358,142.521 223.622,139.067 228.886,133.511 234.151,128.706 239.415,125.403 244.679,122.1 
 249.944,118.946 255.208,116.544 260.472,113.991 265.737,110.538 271.001,105.132 276.265,101.528 281.53,98.5253 286.794,96.5733 292.058,94.7715 297.323,91.7684 
 302.587,88.7653 307.852,85.612 313.116,81.5578 318.38,78.5547 323.645,76.7529 328.909,74.3504 334.173,72.6987 339.438,70.7467 344.702,68.3442 349.966,66.3922 
 355.231,64.8906 360.495,63.0888 365.759,61.1368 371.024,59.1847 376.288,56.9324 381.552,54.8302 386.817,53.0284 392.081,51.5268 397.345,48.3736 402.61,46.4216 
 407.874,43.4185 413.138,40.1151 418.403,39.3643 423.667,37.5624 428.931,36.211 434.196,35.0098 439.46,33.3581 444.724,32.1569 449.989,31.7064 455.253,30.2048 
 460.517,28.5531 465.782,28.1027 471.046,27.2017 476.31,25.55 481.575,23.8983 486.839,22.6971 492.103,21.4959 497.368,20.7451 502.632,19.8441 507.897,18.7931 
 513.161,18.7931 518.425,18.7931 523.69,18.0423 528.954,16.9912 534.218,16.5407 539.483,15.6398 544.747,16.2404 550.011,15.1893 555.276,14.4386 560.54,13.5376 
 565.804,13.3875 571.069,12.937 576.333,12.3364 581.597,10.8348 586.862,10.0841 592.126,9.78376 
 "/>
 
 
 
 
 training set 
 
 
 
 test set

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 0.70 
 
 
 0.75 
 
 
 0.80 
 
 
 accuracy 
 
 
 iteration 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,358.638 57.7953,313.266 76.2205,277.287 94.6457,242.247 113.071,203.233 131.496,168.265 149.921,133.369 168.346,111.261 186.772,92.1874 205.197,76.2206 
 223.622,60.6151 242.047,49.4889 260.472,42.3363 278.898,34.6058 297.323,28.8259 315.748,23.9853 334.173,20.5174 352.598,17.772 371.024,15.1711 389.449,11.9922 
 407.874,12.2089 426.299,11.1974 444.724,10.4749 463.15,9.68022 481.575,9.82471 500,9.82471 518.425,9.53572 536.85,8.95774 555.276,9.10223 573.701,8.88549 
 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 57.7953,318.439 76.2205,282.409 94.6457,250.431 113.071,209.047 131.496,165.638 149.921,130.187 168.346,105.154 186.772,88.8036 205.197,72.1634 
 223.622,57.9831 242.047,49.7353 260.472,41.1982 278.898,36.1338 297.323,30.3459 315.748,27.4519 334.173,24.2686 352.598,21.9534 371.024,19.7829 389.449,18.7701 
 407.874,17.0337 426.299,16.3102 444.724,15.7314 463.15,15.442 481.575,14.7185 500,14.2844 518.425,13.5609 536.85,13.1268 555.276,11.6799 573.701,12.6928 
 592.126,12.2587 
 "/>
 
 
 
 
 training set 
 
 
 
 test set

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 accuracy 
 
 
 iteration 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,360.349 47.381,314.388 55.392,272.991 63.4029,237.95 71.4139,206.936 79.4249,180.576 87.4358,156.812 95.4468,136.852 103.458,120.025 111.469,109.956 
 119.48,100.11 127.491,91.2043 135.502,83.283 143.512,76.57 151.523,70.3046 159.534,65.0237 167.545,60.5484 175.556,56.1178 183.567,51.553 191.578,47.928 
 199.589,45.2876 207.6,41.9311 215.611,39.5592 223.622,37.3216 231.633,35.1286 239.644,33.0253 247.655,31.2351 255.666,28.5499 263.677,27.1626 271.688,25.0145 
 279.699,23.9404 287.71,22.2398 295.721,21.1657 303.732,20.3154 311.743,19.5546 319.754,19.107 327.764,18.033 335.775,17.3169 343.786,16.1086 351.797,15.8401 
 359.808,14.9003 367.819,14.1394 375.83,14.229 383.841,13.8709 391.852,13.2891 399.863,13.1101 407.874,12.4388 415.885,11.9018 423.896,11.3648 431.907,10.5592 
 439.918,10.604 447.929,10.4697 455.94,10.7382 463.951,10.7382 471.962,10.2907 479.973,10.2012 487.984,9.93266 495.995,9.52988 504.005,9.17186 512.016,8.90334 
 520.027,8.94809 528.038,8.3663 536.049,8.32155 544.06,8.3663 552.071,8.54531 560.082,8.27679 568.093,8.50056 576.104,8.05303 584.115,7.87402 592.126,8.00828 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 47.381,321.806 55.392,281.92 63.4029,245.709 71.4139,212.367 79.4249,184.76 87.4358,163.876 95.4468,143.889 103.458,128.741 111.469,115.565 
 119.48,105.706 127.491,96.4739 135.502,88.676 143.512,82.9396 151.523,76.3965 159.534,70.0327 167.545,65.4615 175.556,60.98 183.567,57.574 191.578,52.4651 
 199.589,49.6865 207.6,47.8042 215.611,45.5635 223.622,43.5916 231.633,40.5441 239.644,38.7515 247.655,37.3174 255.666,35.7937 263.677,34.7181 271.688,33.9114 
 279.699,32.7462 287.71,31.3121 295.721,31.1329 303.732,29.6091 311.743,28.7128 319.754,27.0098 327.764,26.2032 335.775,25.2172 343.786,25.2172 351.797,24.4106 
 359.808,23.2453 367.819,22.8868 375.83,23.7831 383.841,22.9765 391.852,23.2453 399.863,22.6179 407.874,22.7076 415.885,21.4527 423.896,21.1838 431.907,20.5564 
 439.918,19.6601 447.929,19.7497 455.94,19.1223 463.951,18.8534 471.962,19.0327 479.973,18.6742 487.984,17.6882 495.995,17.6882 504.005,17.4193 512.016,16.6127 
 520.027,16.6127 528.038,16.6127 536.049,16.2541 544.06,16.1645 552.071,15.806 560.082,15.806 568.093,15.7163 576.104,15.1786 584.115,15.4475 592.126,15.0889 
 
 "/>
 
 
 
 
 training set 
 
 
 
 test set

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 accuracy 
 
 
 iteration 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,358.777 56.6437,302.338 73.9173,244.19 91.1909,188.233 108.465,140.558 125.738,100.683 143.012,74.2598 160.285,58.6164 177.559,47.1797 194.833,37.5833 
 212.106,29.5644 229.38,23.4298 246.654,19.3108 263.927,15.1042 281.201,12.9132 298.474,11.1604 315.748,9.2324 333.022,9.10095 350.295,8.44366 367.569,7.87402 
 384.843,8.48748 402.116,8.92567 419.39,9.01331 436.663,9.18859 453.937,9.05713 471.211,9.32004 488.484,9.32004 505.758,9.88969 523.031,9.71441 540.305,9.80205 
 557.579,9.97733 574.852,10.2402 592.126,9.84587 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 56.6437,310.845 73.9173,253.714 91.1909,199.478 108.465,148.138 125.738,108.207 143.012,81.4403 160.285,65.2924 177.559,54.4979 194.833,45.2831 
 212.106,36.5071 229.38,31.5925 246.654,27.6433 263.927,23.6941 281.201,19.6571 298.474,17.1998 315.748,16.4977 333.022,14.4793 350.295,13.3384 367.569,12.724 
 384.843,13.0751 402.116,13.3384 419.39,13.1628 436.663,12.9873 453.937,14.1282 471.211,13.3384 488.484,12.8118 505.758,13.3384 523.031,13.0751 540.305,12.8996 
 557.579,12.1097 574.852,11.5832 592.126,11.7587 
 "/>
 
 
 
 
 training set 
 
 
 
 test set

final accuracies: train: 0.8189443670844109, test: 0.8169322709163347


...

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 accuracy 
 
 
 iteration 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,367.245 56.1203,332.262 72.8704,268.891 89.6206,205.348 106.371,152.774 123.121,110.65 139.871,77.9398 156.621,55.8528 173.372,41.1528 190.122,31.0728 
 206.872,24.7975 223.622,20.0787 240.372,16.6693 257.122,15.0881 273.873,12.914 290.623,11.9752 307.373,10.8881 324.123,10.1964 340.873,9.67754 357.623,9.13401 
 374.374,9.03519 391.124,8.81284 407.874,8.2199 424.624,8.04696 441.374,8.12107 458.125,7.87402 474.875,8.04696 491.625,7.99755 508.375,7.87402 525.125,7.99755 
 541.875,8.04696 558.626,8.04696 575.376,8.04696 592.126,7.94813 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 56.1203,334.164 72.8704,272.363 89.6206,208.583 106.371,154.302 123.121,115.757 139.871,86.0686 156.621,63.159 173.372,45.247 190.122,34.3613 
 206.872,27.434 223.622,22.5354 240.372,19.5666 257.122,16.6967 273.873,14.2227 290.623,13.0352 307.373,12.095 324.123,11.5013 340.873,10.8085 357.623,10.1653 
 374.374,10.3632 391.124,9.91788 407.874,10.1653 424.624,9.57152 441.374,9.47255 458.125,9.32411 474.875,8.82931 491.625,8.87879 508.375,8.87879 525.125,8.77983 
 541.875,8.87879 558.626,8.87879 575.376,8.77983 592.126,8.82931 
 "/>
 
 
 
 
 training set 
 
 
 
 test set

final accuracies: train: 0.8183807439824945, test: 0.8163346613545817


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 0.65 
 
 
 0.70 
 
 
 0.75 
 
 
 accuracy 
 
 
 iteration 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,363.121 45.3783,317.364 51.3865,281.412 57.3947,251.997 63.4029,229.935 69.4112,211.33 75.4194,193.354 81.4276,180.909 87.4358,168.904 93.444,160.105 
 99.4522,151.745 105.46,146.34 111.469,141.186 117.477,135.906 123.485,130.312 129.493,125.221 135.502,120.696 141.51,116.422 147.518,112.148 153.526,108.942 
 159.534,103.725 165.543,99.7656 171.551,96.8744 177.559,93.3546 183.567,90.9662 189.575,87.7607 195.584,86.0636 201.592,83.5495 207.6,80.9725 213.608,79.2126 
 219.617,78.0184 225.625,75.6928 231.633,73.1158 237.641,71.0417 243.649,69.3446 249.658,66.7048 255.666,63.5621 261.674,61.1108 267.682,59.2881 273.691,57.6539 
 279.699,57.2139 285.707,54.5741 291.715,52.7513 297.723,51.62 303.732,51.2428 309.74,49.2944 315.748,47.1574 321.756,45.7746 327.764,44.8318 333.773,43.449 
 339.781,42.1291 345.789,41.312 351.797,40.3692 357.806,39.175 363.814,38.1065 369.822,36.6609 375.83,35.8438 381.838,35.0267 387.847,34.1467 393.855,33.3296 
 399.863,32.4497 405.871,31.6954 411.879,30.8784 417.888,29.4956 423.896,28.7413 429.904,27.8614 435.912,27.1071 441.921,25.7244 447.929,24.8444 453.937,24.153 
 459.945,23.2731 465.953,22.456 471.962,21.5132 477.97,20.1304 483.978,19.0619 489.986,17.9934 495.995,17.6163 502.003,16.4849 508.011,15.8564 514.019,15.5421 
 520.027,14.4736 526.036,13.7822 532.044,13.2166 538.052,12.6509 544.06,12.0852 550.068,11.5195 556.077,11.3938 562.085,10.7024 568.093,10.011 574.101,9.31964 
 580.11,8.62826 586.118,8.12543 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 45.3783,325.452 51.3865,286.681 57.3947,257.602 63.4029,231.922 69.4112,212.41 75.4194,197.304 81.4276,185.723 87.4358,177.415 93.444,166.589 
 99.4522,158.155 105.46,148.714 111.469,141.035 117.477,133.86 123.485,128.573 129.493,124.041 135.502,120.265 141.51,115.733 147.518,111.075 153.526,107.047 
 159.534,103.648 165.543,100.249 171.551,98.1093 177.559,96.0952 183.567,92.9482 189.575,91.9411 195.584,89.4235 201.592,87.9129 207.6,85.5211 213.608,83.8847 
 219.617,81.2411 225.625,79.9823 231.633,78.8494 237.641,78.0941 243.649,76.7094 249.658,75.1988 255.666,72.3035 261.674,69.9117 267.682,67.6459 273.691,65.5059 
 279.699,63.24 285.707,61.2259 291.715,59.9671 297.723,58.2047 303.732,56.4424 309.74,54.68 315.748,53.1694 321.756,51.2812 327.764,49.2671 333.773,47.8824 
 339.781,46.2459 345.789,44.6095 351.797,44.8612 357.806,44.1059 363.814,44.3577 369.822,43.98 375.83,42.3436 381.838,41.5883 387.847,41.3365 393.855,40.5812 
 399.863,39.3224 405.871,38.3154 411.879,37.1824 417.888,36.9307 423.896,36.0495 429.904,35.4201 435.912,34.1612 441.921,33.5318 447.929,32.273 453.937,31.8954 
 459.945,31.266 465.953,30.0071 471.962,28.4966 477.97,27.4895 483.978,27.2377 489.986,26.6083 495.995,25.9789 502.003,25.7272 508.011,25.2236 514.019,24.7201 
 520.027,23.2095 526.036,23.0836 532.044,22.5801 538.052,22.0766 544.06,21.573 550.068,21.3213 556.077,21.0695 562.085,20.3142 568.093,20.1883 574.101,19.6848 
 580.11,19.3072 586.118,18.3001 592.126,16.9154 
 "/>
 
 
 
 
 training set 
 
 
 
 test set

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 0.65 
 
 
 0.70 
 
 
 0.75 
 
 
 0.80 
 
 
 accuracy 
 
 
 iteration 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,362.999 57.7953,297.114 76.2205,242.628 94.6457,195.355 113.071,158.709 131.496,133.14 149.921,110.728 168.346,89.3455 186.772,74.5969 205.197,64.0346 
 223.622,53.4723 242.047,46.3878 260.472,40.9778 278.898,36.4695 297.323,32.4121 315.748,28.2902 334.173,26.5513 352.598,23.7175 371.024,20.4973 389.449,18.6295 
 407.874,16.955 426.299,15.0873 444.724,13.7348 463.15,13.284 481.575,12.4467 500,11.0298 518.425,9.80615 536.85,8.96889 555.276,8.58246 573.701,8.58246 
 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 57.7953,302.591 76.2205,246.739 94.6457,200.174 113.071,164.573 131.496,130.392 149.921,108.593 168.346,90.4052 186.772,80.7311 205.197,73.1208 
 223.622,60.222 242.047,52.4827 260.472,45.7753 278.898,41.7766 297.323,38.165 315.748,34.9403 334.173,30.9416 352.598,27.3299 371.024,25.2661 389.449,23.3313 
 407.874,21.9124 426.299,21.3965 444.724,20.4936 463.15,20.6226 481.575,18.9457 500,18.6877 518.425,17.5268 536.85,15.334 555.276,14.1731 573.701,13.7862 
 592.126,13.2702 
 "/>
 
 
 
 
 training set 
 
 
 
 test set

final accuracies: train: 0.8186459783833964, test: 0.8174634794156707
final accuracies: train: 0.8173861149791128, test: 0.8146082337317397


Finally average over the different runs and find that the Newton-Raphson method finds slightly better optima.

In [287]:
av_time = mean(exec_times,1)
av_accs = mean(accuracies,1)
println("mean execution time: $(av_time[1]) (GS), $(av_time[2]) (NR)")
println("mean training accuracy: $(av_accs[1]) (GS), $(av_accs[2]) (NR)")
println("mean test accuracy: $(av_accs[3]) (GS), $(av_accs[4]) (NR)")


mean execution time: 4.371790161700001 (GS), 4.566415940000001 (NR)
mean training accuracy: 0.8006100391220742 (GS), 0.8074928718254757 (NR)
mean test accuracy: 0.7983930942895087 (GS), 0.8081739707835326 (NR)
